<a href="https://colab.research.google.com/github/Computer-CGuy/NeuralNetwork-SAT-Solver/blob/main/NeuralNetwork_SAT_Solver%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Generator

In [ ]:
!apt-get install -qq -y cbmc

Selecting previously unselected package libzip4:amd64.
(Reading database ... 160837 files and directories currently installed.)
Preparing to unpack .../libzip4_1.1.2-1.1_amd64.deb ...
Unpacking libzip4:amd64 (1.1.2-1.1) ...
Selecting previously unselected package minisat.
Preparing to unpack .../minisat_1%3a2.2.1-5build1_amd64.deb ...
Unpacking minisat (1:2.2.1-5build1) ...
Selecting previously unselected package cbmc.
Preparing to unpack .../archives/cbmc_5.6-1_amd64.deb ...
Unpacking cbmc (5.6-1) ...
Setting up minisat (1:2.2.1-5build1) ...
Setting up libzip4:amd64 (1.1.2-1.1) ...
Setting up cbmc (5.6-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



In [ ]:
%%writefile code-template.c

#include <stdio.h>


#define A_start {startA:d}
#define A_stop {stopA:d}

#define B_start {startB:d}
#define B_stop {stopB:d}

#define VAL {val:d}

int main(){{
	short int a = nondet_uint();
	short int b = nondet_uint();

	__CPROVER_assume(a<=A_stop&&b<=B_stop);
	__CPROVER_assume(a>=A_start&&b>=B_start);

	short int c = (a+b);
	assert(c!=VAL);
}}

Writing code-template.c


In [ ]:
import numpy as np
import os
import random
os.system("mkdir -p tmp > /dev/null")
file = open("code-template.c","r").read()
i = 0

def decodeSAT(sat):
    if(sat):
        return "SAT"
    else:
        return "UNSAT"
def parse_first_line(s):
    if("p cnf" in s):
        
        variables,clauses = map(int,s.split(" ")[2:])
        # a = torch.zeros((max(variables,clauses)+1, max(variables,clauses)+1))
        a = torch.zeros((800,800))
        # inv = torch.zeros((max(variables,clauses)+1, max(variables,clauses)+1),dtype=bool)
        inv = torch.zeros((800,800),dtype=bool)
        # b = np.zeros(max(variables,clauses)+1)
        return a,inv

def parse_line(s,cc,a,inv):
        
    if("c" in s):
            return
            
    vals = map(int,s.split(" ")[:-1])
    for x in vals:
        if(x<0):
                inv[cc][abs(x)]=1
        if(x==0):
                print("Line Includes 0")
                quit()
        a[cc,abs(x)]=1.0
    return True
def parse_cnf(file_name):
    with open(file_name,"r") as f:
            s = f.readline()
            cc=0
            while s!="":
                if("p cnf" in s):
                    a,inv = parse_first_line(s)
                s.replace("\n", "")
                if(parse_line(s,cc,a,inv)==True):
                    cc+=1
                s = f.readline()
    return a,inv
def gen(i):
    # global i
    # i+=1
    if(i>(DATASET_SIZE//2)):
        sat=True
    else:
        sat=False
    writeFileName = "tmp/code-tmp"+str(i)+".c"
    writeFile = open(writeFileName,"w")

    nums = np.random.randint(low=0,high=1000,size=(4,),dtype=np.uint32)
    nums = np.sort(nums)

    rnp = (nums[0]+nums[1])
    rns = (nums[2]+nums[3])
    val = random.randrange(rnp,rns)
    if(not sat):
        while((val>=rnp) and (val<=rns)):
            val = random.randrange(0,10000)

    # print(nums[0],nums[1],nums[2],nums[3],val,sat)
    txt2 = (file.format(startA=nums[0],startB=nums[1],stopA=nums[2],stopB=nums[3],val=val))


    writeFile.write(txt2)
    writeFile.close()
    os.system("cbmc "+writeFileName+" --dimacs --outfile tmp/"+decodeSAT(sat)+"_"+str(i)+".cnf > /dev/null")
    os.system("rm -rf "+writeFileName+ " > /dev/null")
    a,inv = parse_cnf("tmp/"+decodeSAT(sat)+"_"+str(i)+".cnf")
    a[inv]*=(-1)
    return a
# gen(0)

In [ ]:
from torch.utils.data import Dataset, DataLoader
class DatasetGenerator(Dataset):
    def __init__(self,transform,test=False):        
        self.transform = transform
        self.test=test
    def __len__(self):
        if(self.test):
            return TEST_SIZE
        return DATASET_SIZE

    def __getitem__(self, idx):
        if(idx>(DATASET_SIZE//2)):
            target = 1
        else:
            target = 0
        sample = gen(idx)
        sample = self.transform(sample)
        return sample.unsqueeze(0),target

## Model

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
DATASET_SIZE = 100
TEST_SIZE = 10

In [ ]:
transform = transforms.Compose([])

batch_size = 4

trainset = DatasetGenerator(transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = DatasetGenerator(transform=transform,test=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [ ]:
import matplotlib.pyplot as plt
def imshow(img):
    npimg = img.numpy()
    plt.imshow(npimg)
    plt.show()

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool1 = nn.MaxPool2d(4, 4)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(4, 4)
        self.conv3 = nn.Conv2d(16, 32, 5)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(32, 64, 5)
        self.pool4 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(5184, 5184//2)
        self.fc2 = nn.Linear(5184//2, 5184//4)
        self.fc3 = nn.Linear(5184//4, 2)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        # print(x.shape)
        x = self.pool2(F.relu(self.conv2(x)))
        # print(x.shape)
        x = self.pool3(F.relu(self.conv3(x)))
        # print(x.shape)
        x = self.pool4(F.relu(self.conv4(x)))
        # print(x.shape)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        # x = x.view(-1)
        # print(x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # x = F.relu(self.fc3(x))
        # x = F.relu(self.fc4(x))
        # x = F.relu(self.fc5(x))
        x = self.fc3(x)
        return x

net = Net().to(device)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [ ]:
for epoch in range(100):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0

print('Finished Training')

[1,    10] loss: 0.694
[1,    20] loss: 0.692
[2,    10] loss: 0.709
[2,    20] loss: 0.698
[3,    10] loss: 0.696
[3,    20] loss: 0.701
[4,    10] loss: 0.689
[4,    20] loss: 0.696
[5,    10] loss: 0.694
[5,    20] loss: 0.698
[6,    10] loss: 0.699
[6,    20] loss: 0.696
[7,    10] loss: 0.694
[7,    20] loss: 0.696
[8,    10] loss: 0.694
[8,    20] loss: 0.692
[9,    10] loss: 0.703
[9,    20] loss: 0.694
[10,    10] loss: 0.695
[10,    20] loss: 0.690
[11,    10] loss: 0.694
[11,    20] loss: 0.700
[12,    10] loss: 0.698
[12,    20] loss: 0.693
[13,    10] loss: 0.697
[13,    20] loss: 0.704
[14,    10] loss: 0.689
[14,    20] loss: 0.700
[15,    10] loss: 0.695
[15,    20] loss: 0.694
[16,    10] loss: 0.691
[16,    20] loss: 0.706
[17,    10] loss: 0.684
[17,    20] loss: 0.716
[18,    10] loss: 0.699
[18,    20] loss: 0.699
[19,    10] loss: 0.683
[19,    20] loss: 0.719
[20,    10] loss: 0.702
[20,    20] loss: 0.697
[21,    10] loss: 0.694
[21,    20] loss: 0.711
[22,    10

KeyboardInterrupt: ignored